In [1]:
! pip install pandas matplotlib seaborn scikit-learn

zsh:1: command not found: pip


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ORIGINAL_DATASET_PATH = "../dataset/fake_job_postings.csv"

In [ ]:
df = pd.read_csv(ORIGINAL_DATASET_PATH)